This .ipynb contains preprocessing codes for GEFS reforecast
and GEFS reanalysis datasets


# Importing Libraries

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

c:\Users\bobby\anaconda3\envs\research\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Reanalysis and Reforecast 16 x 19 --> 8 x 11

In [2]:
# reading variable files
PATH = 'C:\\Users\\bobby\\Documents\\GitHub\\climateMedium\\raw_data\\' # change to your own path
ds_reanalysis_tp = xr.open_dataset(PATH + 'GEFSv12-Reanalysis_tp_2000_2019.nc') # only 1 single reanalysis tp

# slicing dimensions 8 x 11 grid points
ds_reanalysis_tp = ds_reanalysis_tp.sel(lon=slice('102.5', '105.00'), lat=('2.5','2.25','2','1.75','1.5','1.25','1','0.75'))

# starting from 2000-01-01 06:00:00 to 2019-12-31 18:00:00, total 29219 time steps
ds_reanalysis_tp = ds_reanalysis_tp.isel(time=slice(1, None)) 
del ds_reanalysis_tp.attrs['history'] # remove long history text

# split into train val test

In [3]:
y_train = ds_reanalysis_tp.sel(time=slice('2000-01-01T06', '2014-01-01T06'))
y_val = ds_reanalysis_tp.sel(time=slice('2014-01-01T12', '2016-12-31T12'))
y_test = ds_reanalysis_tp.sel(time=slice('2016-12-31T18', '2019-12-31T18'))

In [4]:
def timecheck(train, val, test):
    '''
    Return missing time step in train, val and test split
    '''
    train_timecheck = np.arange(np.datetime64("2000-01-01T06"), np.datetime64("2014-01-01T12"), np.timedelta64(6, "h"))
    val_timecheck = np.arange(np.datetime64("2014-01-01T12"), np.datetime64("2016-12-31T18"), np.timedelta64(6, "h"))
    test_timecheck = np.arange(np.datetime64("2016-12-31T18"), np.datetime64("2020-01-01T00"), np.timedelta64(6, "h"))
    print('train:', set(train_timecheck) - set(train.time.values.astype('datetime64[h]')))
    print('val:', set(val_timecheck) - set(val.time.values.astype('datetime64[h]')))
    print('test:', set(test_timecheck) - set(test.time.values.astype('datetime64[h]')))
    return

timecheck(y_train, y_val, y_test) # reanalysis are all ok, no missing dates

train: set()
val: set()
test: set()


# Log transform and normalization

In [5]:
# log transform and min_max normalization for reanalysis_tp TRAINING DATASET
scaler_train_tp = MinMaxScaler() 
y_one_col = y_train.tp.values.reshape([y_train.tp.values.shape[0]*y_train.tp.values.shape[1]*y_train.tp.values.shape[2], 1])
y_one_col = np.log10(y_one_col+1) # 10**X_one_col - 1 to scale back
y_one_col_res = scaler_train_tp.fit_transform(y_one_col) # scaler_train_apcp.inverse_transform(X_one_col_res) to scale back, or use 10**scaler_train_apcp.inverse_transform(X_one_col_res) -1 only
y_train.tp.values = y_one_col_res.reshape(y_train.tp.values.shape)

In [6]:
def transform_val_test(val_test, scaler_train, is_prec=True):
    '''
    Input (example): ds_val_apcp.tp, scaler_train_apcp, True/False
    Output: Transformed validation/test XR data
    If is_prec set to True, variable is precipitation
    '''
    if is_prec == True:
        X_one_col = val_test.values.reshape([val_test.values.shape[0]*val_test.values.shape[1]*val_test.values.shape[2], 1])
        X_one_col = np.log10(X_one_col+1) 
        X_one_col_res = scaler_train.transform(X_one_col) 
        val_test.values = X_one_col_res.reshape(val_test.values.shape)
        return val_test.values
        
    else:
        X_one_col = val_test.values.reshape([val_test.values.shape[0]*val_test.values.shape[1]*val_test.values.shape[2], 1])
        # X_one_col = np.log10(X_one_col+1) 
        X_one_col_res = scaler_train.transform(X_one_col) 
        val_test.values = X_one_col_res.reshape(val_test.values.shape)
        return val_test.values

# reanalysis
y_val.tp.values = transform_val_test(y_val.tp, scaler_train_tp, True)
y_test.tp.values = transform_val_test(y_test.tp, scaler_train_tp, True)

def inverse_val_test(transformed_vt, scaler_train, is_prec=True):
    '''
    Input (example): ds_val_apcp.tp, scaler_train_apcp, True/False
    Output: Inversed of transformed validation/test XR data
    If is_prec set to True, variable is precipitation
    '''
    if is_prec == True:
        X_one_col = transformed_vt.values.reshape([transformed_vt.values.shape[0]*transformed_vt.values.shape[1]*transformed_vt.values.shape[2], 1])
        X_one_col_res = 10**scaler_train.inverse_transform(X_one_col) -1
        transformed_vt.values = X_one_col_res.reshape(transformed_vt.values.shape)
        return transformed_vt.values
    
    else:
        X_one_col = transformed_vt.values.reshape([transformed_vt.values.shape[0]*transformed_vt.values.shape[1]*transformed_vt.values.shape[2], 1])
        X_one_col_res = scaler_train.inverse_transform(X_one_col)
        transformed_vt.values = X_one_col_res.reshape(transformed_vt.values.shape)
        return transformed_vt.values

# retrieving back original precipitation 
# do not use this yet
# ds_val_apcp.tp.values = inverse_val_test(ds_val_apcp.tp, scaler_train_apcp, True) 

# converting reanalysis to classification

In [7]:
quantile_50 = np.quantile(y_train.tp.values, 0.5)
quantile_75 = np.quantile(y_train.tp.values, 0.75)
quantile_95 = np.quantile(y_train.tp.values, 0.95)

y_train.tp.values= np.array(pd.cut(y_train.tp.values.reshape(-1), 
                                bins=[-0.1, 
                                quantile_50,
                                quantile_75,
                                quantile_95,
                                1.1], 
                                labels=[0,1,2,3])).reshape(
                                y_train.tp.shape[0], 
                                y_train.tp.shape[1], 
                                y_train.tp.shape[2])

y_val.tp.values= np.array(pd.cut(y_val.tp.values.reshape(-1), 
                                bins=[-0.1, 
                                quantile_50,
                                quantile_75,
                                quantile_95,
                                1.1], 
                                labels=[0,1,2,3])).reshape(
                                y_val.tp.shape[0], 
                                y_val.tp.shape[1], 
                                y_val.tp.shape[2])

y_test.tp.values= np.array(pd.cut(y_test.tp.values.reshape(-1), 
                                bins=[-0.1, 
                                quantile_50,
                                quantile_75,
                                quantile_95,
                                1.1], 
                                labels=[0,1,2,3])).reshape(
                                y_test.tp.shape[0], 
                                y_test.tp.shape[1], 
                                y_test.tp.shape[2])

In [8]:
# training FOR REANALYSIS CLASSIFICATION
y_class_train = y_train.tp.values
y_class_train = y_class_train[..., np.newaxis]
print(y_class_train.shape)
np.save('C:\\Users\\bobby\\Documents\\GitHub\\climateMedium\\postprocessed_data\\y_class_train.npy', y_class_train)

# val
y_class_val = y_val.tp.values
y_class_val = y_class_val[..., np.newaxis]
print(y_class_val.shape)
np.save('C:\\Users\\bobby\\Documents\\GitHub\\climateMedium\\postprocessed_data\\y_class_val.npy', y_class_val)

# testing
y_class_test = y_test.tp.values
y_class_test = y_class_test[..., np.newaxis]
print(y_class_test.shape)
np.save('C:\\Users\\bobby\\Documents\\GitHub\\climateMedium\\postprocessed_data\\y_class_test.npy', y_class_test)

(20457, 8, 11, 1)
(4381, 8, 11, 1)
(4381, 8, 11, 1)
